In [2]:
import pandas as pd
import numpy as np
import json
import pandas as pd
import re
from rdkit import Chem
from rdkit.Chem import AllChem
from collections import Counter,defaultdict
from ast import literal_eval
import cobra
from cobra import Metabolite,Reaction,Gene
from tqdm import tqdm
import random
import pickle
from multiprocessing import Pool

import sys
# sys.path.append('../Script/')
from common import *
# from check_newinfo_getyeast8U import *

# get Yeast-MetaTwin without lipids

In [3]:
### input
yeast870_path =  '../../Data/model/Yeast-MetaTwin.yml'
YMDB_total_smiles_path = '../../Data/database/ymdb/ymdb_final_result.xlsx'

cut_off = 0.3
num = 'all'
num_process = 5
cut_off_path = f'../../Results/lipid/top{num}_{cut_off}_re/'
metabolites_info_to_GEM_path = cut_off_path + f'metabolites_info_to_GEM_top{num}_{cut_off}_del.csv'
rxndb_total_info_to_model_path = cut_off_path + f'rxndb_total_info_to_model_top{num}_{cut_off}_del.csv'
yeast8_reaction_in_rxndb_json = cut_off_path + f'yeast8_reaction_in_rxndb_top{num}_{cut_off}.json'

### output
yeast8U_withlipids_path = '../../Data/model/Yeast-MetaTwin-lipid.yml'
rxndb_total_info_to_model_reset_path = cut_off_path + f'rxndb_total_info_to_model_top{num}_{cut_off}_del_reset.csv'

# met_info add YMDBID and YMDBname

In [4]:
met_info = pd.read_csv(metabolites_info_to_GEM_path)
met_info = met_info[~met_info['ID'].str.contains('s_')]
YMDB_Data = pd.read_excel(YMDB_total_smiles_path)
YMDB_Data = YMDB_Data[YMDB_Data['in_model']==0]

def process_metinfoID(metinfoID, met_info, YMDB_Data):
    metinfoID_smiles = met_info[met_info['ID'] == metinfoID]['new_met_smiles'].to_list()[0]
    YMDB_ID = []
    for index, row in YMDB_Data.iterrows():
        if compare_smiles_inchikey(metinfoID_smiles, row['SMILES']):
            YMDB_ID.append(row['ID'])
    return metinfoID, YMDB_ID

metinfoID_2_YMDBID = {}
metinfoID_list = list(set(met_info['ID'].to_list()))
process_partial = partial(process_metinfoID, met_info=met_info, YMDB_Data=YMDB_Data)
with mp.Pool(100) as pool:
    results = list(tqdm(pool.imap_unordered(process_partial, metinfoID_list), total=len(metinfoID_list)))
for metinfoID, YMDB_ID in results:
    metinfoID_2_YMDBID[metinfoID] = YMDB_ID

100%|██████████| 12307/12307 [1:56:31<00:00,  1.76it/s] 


# reset rxnID

In [10]:
rxndb_to_model = pd.read_csv(rxndb_total_info_to_model_path)
rxndb_to_model['equation_dict'] = rxndb_to_model['equation_dict'].apply(lambda x:literal_eval(x))
rxndb_to_model = rxndb_to_model[~rxndb_to_model['equation'].str.startswith(' =>')]
rxndb_to_model = rxndb_to_model[~rxndb_to_model['equation'].str.endswith(' => ')]
rxndb_to_model = rxndb_to_model.drop_duplicates(['equation'], keep='first')
with open(yeast8_reaction_in_rxndb_json, 'r') as file:
    yeast8_reaction_in_rxndb_dict = json.load(file)
yeast8_reaction_in_rxndb = list(set([x for lst in yeast8_reaction_in_rxndb_dict.values() for x in lst]))
rxndb_to_model = rxndb_to_model[~rxndb_to_model['NO'].isin(yeast8_reaction_in_rxndb)]
rxndb_to_model = rxndb_to_model.reset_index(drop=True)
rxndb_to_model['NO'] = 'rxnl' + (rxndb_to_model.index + 1).astype(str)
rxndb_to_model.to_csv(rxndb_total_info_to_model_reset_path,index=None)
print(rxndb_to_model.shape)
rxndb_to_model.head(3)

(51340, 8)


,NO,EC_number,templateID,rxn_smiles_basic,rxn_smiles_final,GPR,equation_dict,equation
0,rxnl1,1.1.1,MNXR181206,CC(C)C1CC=C(C=O)CC1>>CC1=CCC(C(C)(C)O)CC1,CC(C)C1CC=C(C=O)CC1.[H+].NC(=O)C1=CN([C@@H]2O[...,(YDR536W) or (YLR174W) or (YHR183W) or (YJL214...,"{'sl_12189': -1, 's_0794': -1, 's_1212': -1, '...",sl_12189 + s_0794 + s_1212 => sl_2404 + s...
1,rxnl2,3.6.1,MNXR152012_reverse,CCCC=CCCCCCCCCCC(=O)OCC(COP(=O)(O)OP(=O)(O)OCC...,CCCC=CCCCCCCCCCC(=O)OCC(COP(=O)(O)OP(=O)(O)OCC...,(YKR053C) or (YJL134W) or (YIL106W) or (YLR447...,"{'sl_9599': -1, 's_0803': -1, 'sl_12165': 1, '...",sl_9599 + s_0803 => sl_12165 + s_0526 + 2 ...
2,rxnl3,3.6.1,MNXR152012_reverse,CCC=CCCCCCCCCCC(=O)OCC(COP(=O)(O)OP(=O)(O)OCC1...,CCC=CCCCCCCCCCC(=O)OCC(COP(=O)(O)OP(=O)(O)OCC1...,(YKR053C) or (YJL134W) or (YIL106W) or (YLR447...,"{'sl_9953': -1, 's_0803': -1, 'sl_4914': 1, 's...",sl_9953 + s_0803 => sl_4914 + s_0526 + 2 s...


In [11]:
get_yeast8U_step4(metabolites_info_to_GEM_path,rxndb_total_info_to_model_reset_path,yeast870_path,yeast8U_withlipids_path,metinfoID_2_YMDBID)

(37751, 4)
(51340, 8)


100%|██████████| 51340/51340 [11:45<00:00, 72.73it/s] 


In [13]:
yeast8U_mod = cobra.io.load_yaml_model(yeast8U_withlipids_path)
yeast8U_mod.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
s_0565,r_1714,1,6,100.00%
s_0925,r_1861,11.58,0,0.00%
s_1277,r_1992,1000,0,0.00%
s_1374,r_2020,0.6365,0,0.00%
s_1438,r_2049,0.6961,0,0.00%
s_1468,r_2060,3.507,0,0.00%
s_4200,r_4593,0.2262,0,0.00%
s_4201,r_4594,0.1156,0,0.00%
s_4202,r_4595,0.4787,0,0.00%
s_4203,r_4596,0.1312,0,0.00%
